In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import category_encoders as ce
from sklearn.model_selection import train_test_split

from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.pipeline import Pipeline

from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

from sklearn.model_selection import StratifiedKFold,train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.multioutput import RegressorChain
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_regression

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, RationalQuadratic
import numpy as np

from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from scipy.stats import uniform, randint

import time

In [ ]:
df = pd.read_csv('cqd_raw.csv').reset_index(drop=True)
df.head()

In [ ]:
corr_matrix = df.corr(method='pearson')

plt.figure(figsize=(6, 4))
sns.heatmap(corr_matrix,
            annot=True,
            fmt='.2f',
            cmap='coolwarm',
            vmin=-1, vmax=1,
            annot_kws={'size': 12, 'color': 'black'})
plt.title('Correlation Matrix')
plt.show()

In [ ]:
abs(df.corr(method='pearson')['Photoluminescence Quantum Yield']).sort_values(ascending=False)

In [ ]:
X = df.drop(columns=['Photoluminescence Quantum Yield'])
y = df['Photoluminescence Quantum Yield']

In [ ]:
def evaluate_default_model(name, model, X, y, outer_cv):
    r2_list, rmse_list, mse_list, mae_list = [], [], [], []

    for train_idx, test_idx in outer_cv.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        r2_list.append(r2_score(y_test, y_pred))
        rmse_list.append(np.sqrt(mean_squared_error(y_test, y_pred)))
        mse_list.append(mean_squared_error(y_test, y_pred))
        mae_list.append(mean_absolute_error(y_test, y_pred))

    return {
        "Model": name,
        "R2": r2_list,
        "RMSE": rmse_list,
        "MSE": mse_list,
        "MAE": mae_list
    }

def run_all_models(X, y):
    outer_cv = KFold(n_splits=10, shuffle=True, random_state=42)
    start_time = time.time()

    models = {
        "MLP-R": Pipeline([
            ('sc', StandardScaler()),
            ('reg', MLPRegressor(random_state=42, max_iter=1500))
        ]),
        "SVM-R": Pipeline([
            ('sc', StandardScaler()),
            ('reg', SVR(kernel='rbf'))
        ]),
        "XGBoost-R": Pipeline([
            ('reg', XGBRegressor(random_state=42))
        ]),
        "GP-R": Pipeline([
            ('sc', StandardScaler()),
            ('reg', GaussianProcessRegressor(kernel =  Matern(length_scale=1.0, length_scale_bounds=(1e-05, 100000.0), nu=1.5) +\
                RationalQuadratic(length_scale=1.0, alpha=1.0, length_scale_bounds=(1e-05, 100000.0), alpha_bounds=(1e-05, 100000.0)),n_restarts_optimizer=3))
        ])
    }

    all_results = []
    for name, model in models.items():
        print(f"Evaluating: {name}")
        try:
            result = evaluate_default_model(name, model, X, y, outer_cv)
            all_results.append(result)
        except Exception as e:
            print(f"Model {name} failed: {e}")

    print(f"Execution time: {time.time() - start_time:.2f} seconds")
    return all_results


def plot_results_boxplot(results):
    records = []
    for res in results:
        for i in range(len(res['R2'])):
            records.append({
                'Model': res['Model'],
                'R2': res['R2'][i],
                'RMSE': res['RMSE'][i],
                'MSE': res['MSE'][i],
                'MAE': res['MAE'][i]
            })
    df = pd.DataFrame(records)

    metrics = ['R2', 'RMSE', 'MSE', 'MAE']
    fig, axes = plt.subplots(nrows=4, figsize=(8, 12))
    for i, metric in enumerate(metrics):
        sns.boxplot(x=metric, y='Model', data=df, ax=axes[i], palette='Set2')
        axes[i].set_ylabel(f"{metric}")
        axes[i].grid(True)
    plt.tight_layout()
    plt.show()

# Single Evaluation

In [ ]:
models = {
    "GP": GaussianProcessRegressor(),
    "XGB": XGBRegressor(random_state=42),
    "MLP": Pipeline([
        ('sc', StandardScaler()),
        ('reg', MLPRegressor(random_state=42, max_iter=1500))
    ]),
    "SVM": Pipeline([
        ('sc', StandardScaler()),
        ('reg', SVR())
    ])
}

outer_cv = KFold(n_splits=10, shuffle=True, random_state=42)

for name, model in models.items():
    print(f"\n{name} Model Evaluation")
    start_time = time.time()

    r2_scores, rmse_scores, mse_scores, mae_scores = [], [], [], []
    y_test_all, y_pred_all = [], []

    for train_idx, test_idx in outer_cv.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        y_test_all.extend(y_test.tolist())
        y_pred_all.extend(y_pred.tolist())

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)

        rmse_scores.append(rmse)
        r2_scores.append(r2)
        mse_scores.append(mse)
        mae_scores.append(mae)

    print(f"R²: {np.mean(r2_scores):.3f}")
    print(f"RMSE: {np.mean(rmse_scores):.3f}")
    print(f"MSE: {np.mean(mse_scores):.3f}")
    print(f"MAE: {np.mean(mae_scores):.3f}")
    print(f"Execution time: {time.time() - start_time:.2f} seconds")

## Hyperparameter Optimization

In [ ]:
outer_cv = KFold(n_splits=10, shuffle=True, random_state=42)
inner_cv = KFold(n_splits=10, shuffle=True, random_state=42)

param_space = {
    'n_estimators': [300, 500, 700],
    'learning_rate': [0.1, 0.5, 1.0, 1.5, 2.0],
    'max_depth': [2, 4, 6, 8, 10, 11],
    'reg_alpha': [0.0, 1.0, 2.5, 4.0, 5.0],
    'reg_lambda': [0.0, 1.0, 2.5, 4.0, 5.0]
}

def evaluate_search(method_name, searcher):
    start = time.time()
    y_test_all, y_pred_all = [], []
    r2_scores, rmse_scores, mse_scores, mae_scores = [], [], [], []

    for train_idx, test_idx in outer_cv.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        searcher.fit(X_train, y_train)
        best_model = searcher.best_estimator_
        y_pred = best_model.predict(X_test)

        y_test_all.extend(y_test.tolist())
        y_pred_all.extend(y_pred.tolist())

        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)

        rmse_scores.append(rmse)
        r2_scores.append(r2)
        mse_scores.append(mse)
        mae_scores.append(mae)

    print(f"\n{method_name} Results:")
    print("Best Params:", searcher.best_params_)
    print(f"R²: {np.mean(r2_scores):.3f}, RMSE: {np.mean(rmse_scores):.3f}, "
          f"MSE: {np.mean(mse_scores):.3f}, MAE: {np.mean(mae_scores):.3f}")
    print(f"Execution time: {time.time() - start:.2f} s")


# Grid Search
grid_search = GridSearchCV(
    XGBRegressor(random_state=42),
    param_space,
    cv=inner_cv,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1, verbose=1
)
evaluate_search("GridSearch", grid_search)

# Random Search
rand_search = RandomizedSearchCV(
    XGBRegressor(random_state=42),
    param_space,
    n_iter=25,
    cv=inner_cv,
    scoring='neg_root_mean_squared_error',
    random_state=123,
    n_jobs=-1, verbose=1
)
evaluate_search("RandomSearch", rand_search)

# Bayesian Optimization
bayes_search = BayesSearchCV(
    estimator=XGBRegressor(random_state=42),
    search_spaces=param_space,
    n_iter=25,
    cv=inner_cv,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=123,
    verbose=1
)
evaluate_search("BayesSearch", bayes_search)
